<a href="https://colab.research.google.com/github/eldercamposds/RegressaoPySpark/blob/main/RegresaoPySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark


In [3]:
from pyspark.sql import SparkSession

# Criand uma sessão Spark
spark = SparkSession.builder \
    .appName("Regressao") \
    .getOrCreate()

# Verificando sessão Spark
print(spark.version)


3.5.5


In [4]:
from pyspark.ml.regression import LinearRegression, RandomForestRegressor

In [5]:
from pyspark.ml.evaluation import RegressionEvaluator

In [6]:
from pyspark.ml.feature import VectorAssembler #oneHotEcoder

In [11]:
carros =  spark.read.csv("/content/drive/MyDrive/Colab Notebooks/PySpark/Spark/download/Carros.csv", header=True, inferSchema=True, sep=";")